## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [6]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [8]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, m_test, y_train, m_test = train_test_split(x_train, y_train, test_size=0.9, random_state=4)
x_test, d_test, y_test, d_test = train_test_split(x_test, y_test, test_size=0.9, random_state=4)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (5000, 32, 32, 3)
5000 train samples
1000 test samples


In [9]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 32, 32, 16)   64          conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 32, 32, 16)   0           batch_normalization_29[0][0]     
__________________________________________________________________________________________________
conv2d_33 

In [10]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 5000 samples, validate on 1000 samples
Epoch 1/30
5000/5000 [==============================] - 108s 22ms/step - loss: 2.4463 - acc: 0.3220 - val_loss: 3.6596 - val_acc: 0.2360
Epoch 2/30
5000/5000 [==============================] - 108s 22ms/step - loss: 2.1014 - acc: 0.4368 - val_loss: 2.5093 - val_acc: 0.3470
Epoch 3/30
5000/5000 [==============================] - 114s 23ms/step - loss: 1.9105 - acc: 0.5064 - val_loss: 2.0687 - val_acc: 0.4240
Epoch 4/30
5000/5000 [==============================] - 126s 25ms/step - loss: 1.7387 - acc: 0.5582 - val_loss: 2.9725 - val_acc: 0.3410
Epoch 5/30
5000/5000 [==============================] - 132s 26ms/step - loss: 1.5670 - acc: 0.6180 - val_loss: 2.7765 - val_acc: 0.3640
Epoch 6/30
5000/5000 [==============================] - 133s 27ms/step - loss: 1.3997 - acc: 0.6760 - val_loss: 4.5218 - val_acc: 0.2470
Epoch 7/30
5000/5000 [==============================] - 131s 26ms/step - loss: 1.2067 - acc: 0.7442 - val_loss: 3.2043 - val_acc: 